Game defined as :

    *State: 
        - Grid size
        - Positions of living cells
    *Rules: eg
        - A cell survives if it has 2 or 3 neighbours -> dies if 1 or 0 neighbours 
        - Dies if it has more than 3 neighbours.
        - If a dead cell has 3 neighbours it becomes alive.
    *Stop test:
        - If current state = previous state 
        

In [19]:
%matplotlib inline
import IPython.display
from io import BytesIO as bio
import PIL.Image
import numpy as np
from abc import ABC, abstractmethod
from copy import copy
import matplotlib.pyplot as plt
import matplotlib
import time
matplotlib.use('GTK3Agg')

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: UserWarning: matplotlib.pyplot as already been imported, this call will have no effect.
  # This is added back by InteractiveShellApp.init_path()


In [20]:
class Game:
    def __init__(self, initial_state, rules,max_size):
        self.initial_state = initial_state
        self.rules = rules
        self.max_size = max_size
    def run_game(self, it):
        state = self.initial_state
        previous_state = None
        progression = []
        i = 0
        while (not state.equals(previous_state) and i < it):
            i += 1
            previous_state = state.copy()
            progression.append(previous_state.grid)
            state = state.apply_rules(self.rules,self.max_size)
        progression.append(state.grid)
        return progression

In [21]:
class State(ABC):
    @abstractmethod
    def copy(self):
        pass

    @abstractmethod
    def apply_rules(self, rules, max_size):
        pass

    @abstractmethod
    def equals(self, other):
        pass

    @abstractmethod
    def get_neighbours(self, elem, max_size):
        pass


class DenseNumpyState(State):
    def __init__(self, grid):
        self.grid = grid

    def copy(self):
        return DenseNumpyState(np.copy(self.grid))

    def equals(self, other):
        if other is None:
            return False
        return np.array_equal(self.grid, other.grid)

    def apply_rules(self, rules, max_size,):
        self.grid = rules.apply_rules(self.grid, max_size, self.get_neighbours)
        return self

    def get_neighbours(self, elem, max_size):
        l = []
        if elem[0]-1 >= 0:
            l.append((elem[0]-1, elem[1]))
        if elem[0]-1 >= 0 and elem[1]-1 >= 0:
            l.append((elem[0]-1, elem[1]-1))
        if elem[0]-1 >= 0 and elem[1]+1 < max_size:
            l.append((elem[0]-1, elem[1]+1))
        if elem[1]-1 >= 0:
            l.append((elem[0], elem[1]-1))
        if elem[1]-1 >= 0 and elem[0]+1 < max_size:
            l.append((elem[0]+1, elem[1]-1))
        if elem[1]+1 < max_size:
            l.append((elem[0], elem[1]+1))
        if elem[0]+1 < max_size:
            l.append((elem[0]+1, elem[1]))
        if elem[1]+1 < max_size and elem[0]+1 < max_size:
            l.append((elem[0]+1, elem[1]+1))
        return l


class SparseSetState(State):
    def __init__(self, grid):
        self.grid = grid

    def copy(self):
        return SparseSetState(copy(self.grid))

    def get_neighbours(self, elem, max_size):
        # Returns the neighbours of a live cell if they lie within the bounds of the grid specified by max_size
        l = []
        if elem[0]-1 >= 0:
            l.append((elem[0]-1, elem[1]))
        if elem[0]-1 >= 0 and elem[1]-1 >= 0:
            l.append((elem[0]-1, elem[1]-1))
        if elem[0]-1 >= 0 and elem[1]+1 < max_size:
            l.append((elem[0]-1, elem[1]+1))
        if elem[1]-1 >= 0:
            l.append((elem[0], elem[1]-1))
        if elem[1]-1 >= 0 and elem[0]+1 < max_size:
            l.append((elem[0]+1, elem[1]-1))
        if elem[1]+1 < max_size:
            l.append((elem[0], elem[1]+1))
        if elem[0]+1 < max_size:
            l.append((elem[0]+1, elem[1]))
        if elem[1]+1 < max_size and elem[0]+1 < max_size:
            l.append((elem[0]+1, elem[1]+1))
        return l

    def equals(self, other):
        if other is None:
            return False
        return self.grid == other.grid

    def apply_rules(self, rules, max_size):
        # Calls the actual rules and provides them with the grid and the neighbour function
        self.grid = rules.apply_rules(self.grid, max_size, self.get_neighbours)
        return self

In [22]:
# %%pixie_debugger
class Rule(ABC):
    @abstractmethod
    def apply_rules(self, grid, max_size, get_neighbours):
        pass


class DenseNumpyRules(Rule):
    def apply_rules(self, grid, max_size, get_neighbours):
        #copied_state = state.copy()
        #grid = state.grid

        grid_ret = copy(grid)
        for i in range(grid.shape[0]):
            for j in range(grid.shape[1]):
                nb = get_neighbours((i, j), max_size)
                counter = 0
                for n in nb:
                    if grid[n] == True:
                        counter += 1
                if (counter < 2 or counter > 3):
                    grid_ret[i][j] = False
                if (counter == 3):
                    grid_ret[i][j] = True
        return grid_ret


class SparseSetRules(Rule):
    def apply_rules(self, grid, max_size, get_neighbours):
        #grid = state.grid
        counter = {}
        for elem in grid:
            if elem not in counter:
                counter[elem] = 0
            nb = get_neighbours(elem, max_size)
            for n in nb:
                if n not in counter:
                    counter[n] = 1
                else:
                    counter[n] += 1
        for c in counter:
            if (counter[c] < 2 or counter[c] > 3):
                grid.discard(c)
            if counter[c] == 3:
                grid.add(c)
        return grid

In [25]:
MAX_ITER = 2000
MAX_SIZE = 200

init = np.zeros((MAX_SIZE, MAX_SIZE), dtype=bool)
# 1 stable blinker - only for dense state
# init[40][40]=True
# init[40][41]=True
# init[40][42]=True
#rules1 = DenseNumpyRules()
#game1 = Game(DenseNumpyState(init), rules1,MAX_SIZE)

#-------------------Sparse examples-------------------#
##### 1 blinker
#board = {(40, 40),(40, 41),(40, 42)}
#####Tetris
#board = {(39, 40),(39, 41),(40, 39),(40, 40),(41, 40)}
##### 4 blinkders
#board={(40, 44),(40, 43),(40, 42),(40, 40),(40, 41)}
##### 4 beehives
#board={(40, 40),(41, 39),(42, 39),(43, 40),(41, 41),(42, 41),(40,41)}
##### 4 distant blocks
#board={(40, 40),(40, 41),(39, 41),(39, 42),(38, 42),(38, 43)}
##### Blaster Canon
board = {(50,180), (51,180), (50,181), (51,181), (60,180), (60,179), (60,181), (61,178),
         (62,177), (63,177), (61,182), (62,183), (63,183), (65,182), (66,181), (66,180),
         (66,179), (65,178), (64,180), (67,180), (70,181), (70,182), (70,183), (71,181),
         (71,182), (71,183), (72,180), (72,184), (74,180), (74,179), (74,184), (74,185),
         (84,182), (84,183), (85,182), (85,183)}

rules2 = SparseSetRules()
game = Game(SparseSetState(board), rules2,MAX_SIZE)
t = time.time()
rw = game.run_game(MAX_ITER)
print(time.time()-t)

1.192842960357666


In [15]:
#transform dense result to numpy array for representation
#res1 = np.array(rw1)
len(rw)

1501

In [16]:
#transform sparse representation to array for plotting
res = np.zeros((len(rw), MAX_SIZE, MAX_SIZE), dtype=bool)
for l in range(0,len(rw)):
    for key in rw[l]:
        res[l,key[0], key[1]] = True 

In [17]:
#transform array to a gif and save to a file
def save_gif(array,file_name):
    array = np.uint8(np.clip(array,0,1)*255.0)
    frames = []
    for frame in range(array.shape[0]):
        img = PIL.Image.fromarray(array[frame])
        img = img.resize((500, 500))
        frames.append(img)
    img.save(file_name, save_all=True, duration=33.33, append_images=frames, loop=0,size=(500,500))

In [18]:
save_gif(res,"small-asdasd.gif")